In [1]:
import requests

from dtos.dtos import *

In [9]:
dataAuth = """
    {
     "grant_type": "client_credentials",
     "client_id": "JTrLV2ByJYJ5w1FU0M8tjkrnOh2U8uD",
     "client_secret": "zEoTATE6iciep+t8iHfbLmmTbJe3k54C2ZpCavXTXe/ZO0FJqbhUsM0prGv/JwgQ",
     "scope": "offers"
    }
"""

# authorize
#########################################################################
cooltraBaseUrl = "https://maas.zeus-stage.cooltra.com"

response = req.request("POST",
                       url=cooltraBaseUrl + "/oauth/token",
                       headers= {"Content-Type" : "application/json", "Accept" : "application/json"},
                       data=dataAuth
                       )

result : CooltraAuthResponseDTO = CooltraAuthResponseDTOSchema().loads(response.text)
token = "Bearer " + result.access_token

#########################################################################


3600

In [6]:
config_url = cooltraBaseUrl + "/maas/v1/configuration"

result = req.request("GET", 
            url=config_url,
            headers={"Content-Type" : "application/json",
                     "Accept" : "application/json", 
                     "Authorization" : token})